# ESG Scorecard

In [1]:
import pandas as pd
import requests, json, math, time
import plotly.offline as offline
import plotly.graph_objects as go
from plotly.subplots import make_subplots
offline.offline.init_notebook_mode(connected=True)

In [2]:
APP_KEY                     = ''
RDP_LOGIN                   = ''
RDP_PASSWORD                = ''

## Platform function defines

In [3]:
tknObject = {}

def getToken():
    global tknObject
    if 'expiry_tm' in tknObject and tknObject['expiry_tm'] > time.time():
        return tknObject["access_token"]

    print('Getting access token...')
    if 'refresh_token' in tknObject:
        tData = {
            'refresh_token': tknObject['refresh_token'],
            'grant_type': "refresh_token",
        }
    else:
        tData = {
            'username': RDP_LOGIN,
            'password': RDP_PASSWORD,
            'grant_type': 'password',
            'scope': 'trapi',
            'takeExclusiveSignOnControl': 'true'
        }
    
    response = requests.post('https://api.refinitiv.com/auth/oauth2/v1/token',
        headers = {'Accept': 'application/json'},
        data = tData,
        auth = (APP_KEY, ''))

    if response.status_code != 200:
        raise Exception('Failed to get access token {0} - {1}'.format(response.status_code, response.text))

    tknObject = json.loads(response.text)
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10
    return tknObject['access_token']


In [4]:
def getRequest(URL, requestData):
    dResp = requests.get(URL, headers = {'Authorization': 'Bearer ' + getToken()}, params = requestData)

    if dResp.status_code != 200:
        raise Exception('Request failed {0} - {1}'.format(dResp.status_code, dResp.text))
    else:
        jResp = json.loads(dResp.text)
        return jResp 

In [5]:
def postRequest(URL, requestData):
    dResp = requests.post(URL, headers = {'Authorization': 'Bearer ' + getToken(),  'Content-Type': 'application/json'}, data = json.dumps(requestData))
    if dResp.status_code != 200:
        raise Exception('Request failed {0} - {1}'.format(dResp.status_code, dResp.text))
    else:
        jResp = json.loads(dResp.text)
        return jResp

## Get data and save 

In [6]:
# get the ESG coverage universe
_jUniv = getRequest('https://api.refinitiv.com/data/environmental-social-governance/v1/universe', {})
masterDF = pd.DataFrame(_jUniv['data'], columns=['Instrument', 'PrimaryRIC', 'Common Name']).set_index('Instrument')
print('ESG Universe item count: %d' % len(masterDF))
display(masterDF.head())

Getting access token...
ESG Universe item count: 9911


,PrimaryRIC,Common Name
Instrument,,
4295533401,RST^J20,Rosetta Stone Inc
4295613014,PWF.TO^B20,Power Financial Corp
4295641240,KE.O,Kimball Electronics Inc
4295856018,PATA.BA,Importadora y Exportadora de la Patagonia SA
4295856019,COME.BA,Sociedad Comercial del Plata SA


In [7]:
# break up ESG universe into buckets and get ESG scores and TRBC code for it
bucketSize = 100
counter = 0
_actualList = masterDF.index.tolist()
listBuckets = [_actualList[i:i + bucketSize] for i in range(0, len(_actualList), bucketSize)]

In [8]:
valuesDF = pd.DataFrame()
for bList in listBuckets:
    counter += 1
    if not (counter % 10):
        print('Getting bucket: %d' % counter)
    _scores = getRequest('https://api.refinitiv.com/data/environmental-social-governance/v1/views/scores-standard?universe=' + ','.join(bList), {"start": 0, "end": 0})
    _tempDF1 = pd.DataFrame(_scores['data'], columns=[d['title'] for d in _scores['headers']]).set_index('Instrument')
    searchReq = {
        "View": "Organisations",
        "Filter": "OAPermID in (\'" + '\' \''.join(bList) + "\')",
        "Select": "OAPermID, TRBCHierarchicalCode, TRBCHierarchicalCodeMeaning",
        "Top": bucketSize*2
    }
    _trbcCodes = postRequest('https://api.refinitiv.com/discovery/search/v1/', searchReq)
    _tempDF2 = pd.DataFrame(_trbcCodes['Hits']).rename(columns={"OAPermID": "Instrument"}).set_index('Instrument')
    valuesDF = valuesDF.append(_tempDF1.join(_tempDF2)) 


Getting bucket: 10
Getting bucket: 20
Getting bucket: 30
Getting bucket: 40
Getting bucket: 50
Getting bucket: 60
Getting bucket: 70
Getting bucket: 80
Getting bucket: 90
Getting bucket: 100


In [9]:
masterDF = masterDF.join(valuesDF)
masterDF.reset_index(inplace=True)
masterDF.set_index('PrimaryRIC', inplace=True)
masterDF.head()

,Instrument,Common Name,Period End Date,ESG Combined Score,ESG Score,ESG Controversies Score,Environment Pillar Score,Social Pillar Score,Governance Pillar Score,Resource Use Score,...,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score,ESG Reporting Scope,ESG Report Auditor Name,ESG Period Last Update Date,TRBCHierarchicalCode,TRBCHierarchicalCodeMeaning
PrimaryRIC,,,,,,,,,,,,,,,,,,,,,
RST^J20,4295533401,Rosetta Stone Inc,2019-12-31,36.037039,36.037039,100.0,3.016334,54.794120,29.791320,6.170213,...,69.612069,35.456731,35.481185,30.685981,0.000000,NaN,None,2020-09-25T00:00:00,5720102012,Application Software
PWF.TO^B20,4295613014,Power Financial Corp,2018-12-31,50.090897,50.090897,100.0,67.115917,51.089269,43.251122,80.534351,...,69.138756,35.750000,38.235294,25.232198,95.358650,100.0,PricewaterhouseCoopers,2020-06-04T00:00:00,5530101010,Multiline Insurance & Brokers - NEC
KE.O,4295641240,Kimball Electronics Inc,2019-06-30,39.529616,39.529616,100.0,33.089184,51.684797,27.202088,44.402985,...,55.625000,94.520548,27.855478,25.824176,26.002004,100.0,None,2020-07-03T00:00:00,5710101010,Semiconductors - NEC
PATA.BA,4295856018,Importadora y Exportadora de la Patagonia SA,2019-06-30,7.085719,7.085719,100.0,4.443027,7.218541,8.988095,0.000000,...,9.426230,0.000000,6.250000,24.107143,0.000000,NaN,None,2020-10-09T00:00:00,5430102012,Supermarkets & Convenience Stores
COME.BA,4295856019,Sociedad Comercial del Plata SA,2019-12-31,14.168024,14.168024,100.0,0.000000,15.932934,29.821429,0.000000,...,2.654867,70.283019,34.821429,33.035714,0.000000,NaN,None,2020-09-11T00:00:00,5320302010,Construction Supplies & Fixtures - NEC


In [10]:
# save the dataframe for future load
masterDF.to_pickle('./master_dataframe.pkl')

In [11]:
# optionally save as CSV
masterDF.to_csv('./master_dataframe.csv')

## Load saved data and process

In [2]:
masterDF = pd.read_pickle('./master_dataframe.pkl')
masterDF.head()

,Instrument,Common Name,Period End Date,ESG Combined Score,ESG Score,ESG Controversies Score,Environment Pillar Score,Social Pillar Score,Governance Pillar Score,Resource Use Score,...,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score,ESG Reporting Scope,ESG Report Auditor Name,ESG Period Last Update Date,TRBCHierarchicalCode,TRBCHierarchicalCodeMeaning
PrimaryRIC,,,,,,,,,,,,,,,,,,,,,
RST^J20,4295533401,Rosetta Stone Inc,2019-12-31,36.037039,36.037039,100.0,3.016334,54.794120,29.791320,6.170213,...,69.612069,35.456731,35.481185,30.685981,0.000000,NaN,None,2020-09-25T00:00:00,5720102012,Application Software
PWF.TO^B20,4295613014,Power Financial Corp,2018-12-31,50.090897,50.090897,100.0,67.115917,51.089269,43.251122,80.534351,...,69.138756,35.750000,38.235294,25.232198,95.358650,100.0,PricewaterhouseCoopers,2020-06-04T00:00:00,5530101010,Multiline Insurance & Brokers - NEC
KE.O,4295641240,Kimball Electronics Inc,2019-06-30,39.529616,39.529616,100.0,33.089184,51.684797,27.202088,44.402985,...,55.625000,94.520548,27.855478,25.824176,26.002004,100.0,None,2020-07-03T00:00:00,5710101010,Semiconductors - NEC
PATA.BA,4295856018,Importadora y Exportadora de la Patagonia SA,2019-06-30,7.085719,7.085719,100.0,4.443027,7.218541,8.988095,0.000000,...,9.426230,0.000000,6.250000,24.107143,0.000000,NaN,None,2020-10-09T00:00:00,5430102012,Supermarkets & Convenience Stores
COME.BA,4295856019,Sociedad Comercial del Plata SA,2019-12-31,14.168024,14.168024,100.0,0.000000,15.932934,29.821429,0.000000,...,2.654867,70.283019,34.821429,33.035714,0.000000,NaN,None,2020-09-11T00:00:00,5320302010,Construction Supplies & Fixtures - NEC


In [3]:
# count ones which don't have TRBC code
print('Number of items without any TRBC code: %d' % len(masterDF[masterDF['TRBCHierarchicalCode'].isnull()]))

Number of items without any TRBC code: 4


## Plot the quantile chart

In [4]:
def plotQuantilesV2(instrument):
    _tSer = masterDF.loc[instrument]
    _trbcCode = _tSer['TRBCHierarchicalCode']
    _subdf = masterDF[masterDF['TRBCHierarchicalCode'] == _trbcCode]
    _metrics = ['ESG Score', 'Environment Pillar Score', 'Social Pillar Score', 'Governance Pillar Score']
    _binSz = 5
    _bClrs = ['red','yellow','green']

    _traces = {}
    _trNames = []
    for _itemName in _metrics:
        _tSVal = _tSer[_itemName]
        _trNames.append(_itemName + ' (%d)' % _tSVal) 
        _mClrs = ['grey']*(math.ceil(100/_binSz))
        _mClrs[int(_tSVal/_binSz)] = _bClrs[int(_tSVal/33)]
        _traces[_itemName] = go.Histogram(x=_subdf[_itemName],xbins=dict(start=0,end=100,size=_binSz),marker={'color':_mClrs})

    fig = go.Figure()
    fig = make_subplots(rows=2, cols=2, subplot_titles=(_trNames))
    fig.append_trace(_traces[_metrics[0]], 1, 1)
    fig.append_trace(_traces[_metrics[1]], 1, 2)
    fig.append_trace(_traces[_metrics[2]], 2, 1)
    fig.append_trace(_traces[_metrics[3]], 2, 2)

    fig.update_layout(
        title_text= _tSer['Common Name'] + '<br>' + _tSer['TRBCHierarchicalCodeMeaning'],
        bargap=0.2, showlegend=False
    )

    fig.show()

In [5]:
plotQuantilesV2('IBM')

In [6]:
plotQuantilesV2('CF.TO')

In [7]:
plotQuantilesV2('KO')